My data source is https://exoplanetarchive.ipac.caltech.edu/cgi-bin/TblView/nph-tblView?app=ExoTbls&config=cumulative
It is from the NASA DataSet Archive for Exoplanets. 
The two hypotheses I will be testing are :
1. Bias exists in the search for exoplanets;
2. Systems with a G2 or Red Giant star are more likely to have earth-type planets



In [1]:
import pandas as pd
import numpy as np

In [2]:
from scipy import stats

In [3]:
import seaborn as sns 
sns.set()

In [4]:
import matplotlib.pyplot as plt

In [5]:
from sqlalchemy import create_engine

In [6]:
exo=pd.read_excel('/content/drive/MyDrive/Colab Dataset/exoplanet.xlsx')


In [7]:
exo.columns

Index(['rowid', 'kepid', 'kepoi_name', 'kepler_name', 'koi_disposition',
       'koi_vet_stat', 'koi_vet_date', 'koi_pdisposition', 'koi_score',
       'koi_fpflag_nt',
       ...
       'koi_dicco_mdec', 'koi_dicco_mdec_err', 'koi_dicco_msky',
       'koi_dicco_msky_err', 'koi_dikco_mra', 'koi_dikco_mra_err',
       'koi_dikco_mdec', 'koi_dikco_mdec_err', 'koi_dikco_msky',
       'koi_dikco_msky_err'],
      dtype='object', length=141)

In [8]:
exo.shape

(9564, 141)

In [9]:
exo.head(5)

,rowid,kepid,kepoi_name,kepler_name,koi_disposition,koi_vet_stat,koi_vet_date,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_disp_prov,koi_comment,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,koi_time0,koi_time0_err1,koi_time0_err2,koi_eccen,koi_eccen_err1,koi_eccen_err2,koi_longp,koi_longp_err1,koi_longp_err2,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,koi_duration_err1,koi_duration_err2,koi_ingress,koi_ingress_err1,koi_ingress_err2,koi_depth,...,koi_smass_err2,koi_sage,koi_sage_err1,koi_sage_err2,koi_sparprov,ra,dec,koi_kepmag,koi_gmag,koi_rmag,koi_imag,koi_zmag,koi_jmag,koi_hmag,koi_kmag,koi_fwm_stat_sig,koi_fwm_sra,koi_fwm_sra_err,koi_fwm_sdec,koi_fwm_sdec_err,koi_fwm_srao,koi_fwm_srao_err,koi_fwm_sdeco,koi_fwm_sdeco_err,koi_fwm_prao,koi_fwm_prao_err,koi_fwm_pdeco,koi_fwm_pdeco_err,koi_dicco_mra,koi_dicco_mra_err,koi_dicco_mdec,koi_dicco_mdec_err,koi_dicco_msky,koi_dicco_msky_err,koi_dikco_mra,koi_dikco_mra_err,koi_dikco_mdec,koi_dikco_mdec_err,koi_dikco_msky,koi_dikco_msky_err
0,1,10797460,K00752.01,Kepler-227 b,CONFIRMED,Done,2018-08-16,CANDIDATE,1.000,0,0,0,0,q1_q17_dr25_sup_koi,NO_COMMENT,9.488036,2.775000e-05,-2.775000e-05,170.538750,0.002160,-0.002160,2455003.539,0.002160,-0.002160,0.0,NaN,NaN,NaN,NaN,NaN,0.146,0.318,-0.146,2.95750,0.08190,-0.08190,NaN,NaN,NaN,615.8,...,-0.046,NaN,NaN,NaN,q1_q17_dr25_stellar,291.93423,48.141651,15.347,15.890,15.270,15.114,15.006,14.082,13.751,13.648,0.002,19.462294,1.400000e-05,48.14191,0.000130,0.430,0.510,0.940,0.480,-0.00020,0.00032,-0.00055,0.00031,-0.010,0.130,0.200,0.160,0.200,0.170,0.080,0.130,0.310,0.170,0.320,0.160
1,2,10797460,K00752.02,Kepler-227 c,CONFIRMED,Done,2018-08-16,CANDIDATE,0.969,0,0,0,0,q1_q17_dr25_sup_koi,NO_COMMENT,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,-0.003520,2454995.514,0.003520,-0.003520,0.0,NaN,NaN,NaN,NaN,NaN,0.586,0.059,-0.443,4.50700,0.11600,-0.11600,NaN,NaN,NaN,874.8,...,-0.046,NaN,NaN,NaN,q1_q17_dr25_stellar,291.93423,48.141651,15.347,15.890,15.270,15.114,15.006,14.082,13.751,13.648,0.003,19.462265,2.000000e-05,48.14199,0.000190,-0.630,0.720,1.230,0.680,0.00066,0.00065,-0.00105,0.00063,0.390,0.360,0.000,0.480,0.390,0.360,0.490,0.340,0.120,0.730,0.500,0.450
2,3,10811496,K00753.01,NaN,CANDIDATE,Done,2018-08-16,CANDIDATE,0.000,0,0,0,0,q1_q17_dr25_sup_koi,DEEP_V_SHAPED,19.899140,1.494000e-05,-1.494000e-05,175.850252,0.000581,-0.000581,2455008.850,0.000581,-0.000581,0.0,NaN,NaN,NaN,NaN,NaN,0.969,5.126,-0.077,1.78220,0.03410,-0.03410,NaN,NaN,NaN,10829.0,...,-0.121,NaN,NaN,NaN,q1_q17_dr25_stellar,297.00482,48.134129,15.436,15.943,15.390,15.220,15.166,14.254,13.900,13.826,0.278,19.800321,1.900000e-06,48.13412,0.000020,-0.021,0.069,-0.038,0.071,0.00070,0.00240,0.00060,0.00340,-0.025,0.070,-0.034,0.070,0.042,0.072,0.002,0.071,-0.027,0.074,0.027,0.074
3,4,10848459,K00754.01,NaN,FALSE POSITIVE,Done,2018-08-16,FALSE POSITIVE,0.000,0,1,0,0,q1_q17_dr25_sup_koi,MOD_ODDEVEN_DV---MOD_ODDEVEN_ALT---DEEP_V_SHAPED,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,-0.000115,2455003.308,0.000115,-0.000115,0.0,NaN,NaN,NaN,NaN,NaN,1.276,0.115,-0.092,2.40641,0.00537,-0.00537,NaN,NaN,NaN,8079.2,...,-0.077,NaN,NaN,NaN,q1_q17_dr25_stellar,285.53461,48.285210,15.597,16.100,15.554,15.382,15.266,14.326,13.911,13.809,0.000,19.035638,8.600000e-07,48.28521,0.000007,-0.111,0.031,0.002,0.027,0.00302,0.00057,-0.00142,0.00081,-0.249,0.072,0.147,0.078,0.289,0.079,-0.257,0.072,0.099,0.077,0.276,0.076
4,5,10854555,K00755.01,Kepler-664 b,CONFIRMED,Done,2018-08-16,CANDIDATE,1.000,0,0,0,0,q1_q17_dr25_sup_koi,NO_COMMENT,2.525592,3.761000e-06,-3.761000e-06,171.595550,0.001130,-0.001130,2455004.596,0.001130,-0.001130,0.0,NaN,NaN,NaN,NaN,NaN,0.701,0.235,-0.478,1.65450,0.04200,-0.04200,NaN,NaN,NaN,603.3,...,-0.136,NaN,NaN,NaN,q1_q17_dr25_stellar,288.75488,48.226200,15.509,16.015,15.468,15.292,15.241,14.366,14.064,13.952,0.733,19.250326,9.700000e-06,48.22626,0.000100,-0.010,0.350,

As I am working with star radius and temperature, I will get rid of null values for those columns

Hypothesis One: Bias exists in the search for exoplanets. We look at stars that are most like our sun (G2) rather than all options. The G2 average temperature and radius will be the data used. 

Ho mean =temp [5300K, 6000K] AND radius [.84, 1.15]

Ha mean =temp [<5300K,6000K >] OR radius [<.84, >1.15]

***Since the variables we are testing for are temperature (koi_steff), the radius with respect to the sun (koi_srad), and the equilibrium temperature of the planet (koi_teq), I will make a chart with those three factors to help with clarity.***

In [10]:
exo_df=exo.iloc[0:9524, [4,87,96,58]]

In [11]:
exo_df

,koi_disposition,koi_steff,koi_srad,koi_teq
0,CONFIRMED,5455.0,0.927,793.0
1,CONFIRMED,5455.0,0.927,443.0
2,CANDIDATE,5853.0,0.868,638.0
3,FALSE POSITIVE,5805.0,0.791,1395.0
4,CONFIRMED,6031.0,1.046,1406.0
...,...,...,...,...
9519,FALSE POSITIVE,6184.0,1.038,1776.0
9520,CANDIDATE,5999.0,0.967,1037.0
9521,FALSE POSITIVE,4987.0,0.727,226.0
9522,FALSE POSITIVE,6050.0,1.333,336.0


In [12]:
exo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9524 entries, 0 to 9523
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   koi_disposition  9524 non-null   object 
 1   koi_steff        9161 non-null   float64
 2   koi_srad         9161 non-null   float64
 3   koi_teq          9161 non-null   float64
dtypes: float64(3), object(1)
memory usage: 297.8+ KB


***Now we drop the nulls for the star temp and star radius***

In [13]:
exo_df_filtered=exo_df.dropna(axis=0, subset=('koi_srad','koi_steff', 'koi_disposition' ))

In [14]:
exo_df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9161 entries, 0 to 9523
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   koi_disposition  9161 non-null   object 
 1   koi_steff        9161 non-null   float64
 2   koi_srad         9161 non-null   float64
 3   koi_teq          9161 non-null   float64
dtypes: float64(3), object(1)
memory usage: 357.9+ KB


*** suntype gives the parameters for the stars that are like our sun with respect to temperature (koi_steff) and radius (koi_srad)***

In [15]:
suntype_temp= (exo_df_filtered['koi_steff']>= 5300) & (exo_df_filtered['koi_steff']<=6000)

In [16]:
suntype_temp.value_counts()


False    5306
True     3855
Name: koi_steff, dtype: int64

In [17]:
suntype_rad= (exo_df_filtered['koi_srad']>= .84) & (exo_df_filtered['koi_srad']<=1.15)

In [18]:
suntype_rad.value_counts()

False    5565
True     3596
Name: koi_srad, dtype: int64

In [19]:
suntype= ((suntype_rad == True)& (suntype_temp == True))

In [20]:
suntype.value_counts()

False    7026
True     2135
dtype: int64

***This will determine the number of False Positives that were sun-like***

In [21]:
suntype_fp=((suntype) & (exo_df_filtered['koi_disposition']=='FALSE POSITIVE'))

In [22]:
suntype_fp.value_counts()

False    8217
True      944
dtype: int64

***Sun like stars that were confirmed to have planets***

In [23]:
suntype_conf=((suntype) & (exo_df_filtered['koi_disposition']=='CONFIRMED'))

In [24]:
suntype_conf.value_counts()

False    8474
True      687
dtype: int64

***All stars that are confirmed to have planets***

In [25]:
allstar_conf=(exo_df_filtered['koi_disposition']=='CONFIRMED')

In [26]:
allstar_conf.value_counts()

False    6804
True     2357
Name: koi_disposition, dtype: int64

In [27]:
allstar_fp=(exo_df_filtered['koi_disposition']=='FALSE POSITIVE')

In [28]:
allstar_fp.value_counts()

False    4611
True     4550
Name: koi_disposition, dtype: int64

***non_suntype gives the parameters for the stars that are not like our sun with respect to temperature (koi_steff) and radius (koi_srad)***

In [29]:
non_suntype_temp=(exo_df_filtered['koi_steff']< 5300) | (exo_df_filtered['koi_steff']>6000)

In [30]:
non_suntype_temp.value_counts()

True     5306
False    3855
Name: koi_steff, dtype: int64

In [31]:
non_suntype_rad= (exo_df_filtered['koi_srad']<.84) | (exo_df_filtered['koi_srad']>1.15)

In [32]:
non_suntype_rad.value_counts()

True     5565
False    3596
Name: koi_srad, dtype: int64

In [33]:
non_suntype=((non_suntype_rad == True) |(non_suntype_temp == True))

In [34]:
non_suntype.value_counts()

True     7026
False    2135
dtype: int64

In [35]:
non_suntype_fp=((non_suntype) & (exo_df_filtered['koi_disposition']=='FALSE POSITIVE'))

In [36]:
non_suntype_fp.value_counts()

False    5555
True     3606
dtype: int64

Descriptions of suntype and non_suntype

In [37]:
exo_df_filtered.describe()

,koi_steff,koi_srad,koi_teq
count,9161.000000,9161.000000,9161.000000
mean,5705.546447,1.723642,1085.483353
std,790.034188,6.126228,852.836672
min,2661.000000,0.109000,25.000000
25%,5310.000000,0.829000,541.000000
50%,5767.000000,1.000000,879.000000
75%,6110.000000,1.342000,1378.000000
max,11360.000000,229.908000,14667.000000


***If we limit the koi_srad<2 (the star radius is up to twice as large as our sun):***

In [38]:
exo_df_filtered_2=exo_df_filtered[exo_df_filtered.koi_srad<2]

In [39]:
exo_df_filtered_2.describe()

,koi_steff,koi_srad,koi_teq
count,8075.000000,8075.000000,8075.000000
mean,5659.915913,1.013841,958.289164
std,681.660245,0.315442,580.684294
min,2661.000000,0.109000,25.000000
25%,5353.000000,0.810000,522.000000
50%,5765.000000,0.959000,834.000000
75%,6078.000000,1.154000,1266.000000
max,11068.000000,1.999000,4391.000000


***We lose 1046 values, approx. 11.5% of our data, suggesting that we do have a preference in looking for exoplanets. Also noteworthy is that the average temperature is within the G2 range***

In [ ]:
plt.figure(figsize=(40,5))
sns.boxplot(x='koi_srad', y='koi_steff', data = exo_df_filtered_2)
plt.xticks([]) 


## ***This chart shows that we tend to look at certain temperature range that is between 4000K and 6500K.  Another way of saying this is that the temp of a star is the first aspect that is noticed and we hone in on those values closest to our Sun.***

**Compare the False Positives**

In [ ]:
stats.ttest_ind(suntype_rad, non_suntype_rad)

In [ ]:
stats.ttest_ind(suntype_temp, non_suntype_temp)

In [ ]:
stats.ttest_ind(suntype, non_suntype)

In [ ]:
stats.ttest_ind(non_suntype_fp, suntype_fp)

In [ ]:
sns.displot(exo_df_filtered['koi_steff'])

In [ ]:
sns.scatterplot(x= "koi_srad", y='koi_steff', hue=non_suntype_fp, data=exo_df_filtered)

### ***The data indicates that we reject the null that bias exists in looking for exoplanets around stars that resemble our Sun. With p=0 and the graph appearance, it demonstrates bias in the search.It the description we see that the mean temperature is 5706, in the middle of the [5300K, 6000K] range. The radius is averaging 1.72, which is outside the range. It appears that we look for stars with the same temperature as our Sun and then discover the radius after investigation. ***

***I am not assessing the CANDIDATE class for this hypothesis, only the known FALSE POSITIVE and CONFIRMED classes. The below chart gets rid of the CANDIDATE***

In [ ]:
noncan=exo_df_filtered[exo.koi_disposition !='CANDIDATE']
noncan.head()

In [ ]:
noncan.describe()

***The red giants in the study***



In [ ]:
red_giant= (exo_df_filtered['koi_srad']>=100) & (exo_df_filtered['koi_steff']<=5000)

In [ ]:
red_giant.value_counts()

***As only 9 red giants are in the study, I will determine if they have earth-like planets before moving forward

In [ ]:
red_giant_earth=(red_giant & earth_type)

In [ ]:
red_giant_earth.value_counts()

I will no longer consider the red giant class as they have no earth like planets and are otherwise statistically insignificant

***For an earth type planet, I am using a temperature equilibrium of[228K, 339K] which is approximently [-50F, 150F]***

In [ ]:
earth_type=(exo_df_filtered['koi_teq']<339) & (exo_df_filtered['koi_teq']>228)

In [ ]:
earth_type.value_counts()

In [ ]:
exo_df_filtered.describe()

In [ ]:
non_earth_type= (exo_df_filtered['koi_teq']>339) |(exo_df_filtered['koi_teq']<228)

In [ ]:
non_earth_type.value_counts()

In [ ]:
earth_system= (suntype & earth_type)

In [ ]:
earth_system.value_counts()

In [ ]:
earth_non_system= (earth_type & non_suntype)

In [ ]:
earth_non_system.value_counts()

In [ ]:
stats.ttest_ind(earth_system,earth_non_system)

In [ ]:
stats.ttest_ind(earth_type, suntype)

In [ ]:
stats.ttest_ind(non_suntype, earth_type)

In [ ]:
sns.pairplot(exo_df_filtered)

In [ ]:
sns.distplot(exo_df_filtered['koi_teq'])

In [ ]:
sns.scatterplot(x='koi_steff', y= 'koi_teq', data=exo_df_filtered)

In [ ]:
sns.scatterplot(x='koi_srad', y= 'koi_teq', data=exo_df_filtered)

In [ ]:
stats.ttest_ind((exo_df_filtered['koi_teq']<339) & (exo_df_filtered['koi_teq']>228), exo_df_filtered['koi_steff'])

CONCLUSION: We see that most of the earth-like planets as described with the given parameters are within the temperature range of the G2 stars (Sun-like) and so they are likely to be found around them.We therefore reject the null that star type and earth-like conditions are unrelated.

###The Hypo 1 showed that there is bias in looking for planets around G2 stars. In Hypo 2 I showed that the preference is warranted, as the largest possibility of finding an earth-like planet is around G2 stars.

## Therefore I would suggest at prioritizing suntype stars. Once technology updates to include radii information with temperature readings, we can pinpoint and economize our search for extraterrestial life as well as potentially habitable exoplanets. 